# Ejercicio de BYTE-PAIR ENCODING
Integrantes:
*   Aguilar Valenzuela Luis Hector
*   Camargo Loaiza Julio Andres
*   Minjares Neriz Victor Manuel

# Importaciones

In [2]:
import re, collections

# Funciones

### get_vocab(filename)
Función que recibe el nombre de el archivo de texto y devuelve un vocabulario de palabras con la frecuencia de cada palabra y un separador en cada palabra

In [3]:
# Funcion para tokenizar el texto por palabras/espacios.

def get_vocab(filename):
    # La funcion defaultdict crea un dictionario vacio. 
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            # La función strip quita los espacios al principio y al final de un string
            # La función split separa las palabras y las devuelve en un array
            words = line.strip().split()
            
            # Recorre cada palabra del arreglo de palabras
            for word in words:
                # Aqui se llena el diccionario. Agregara al diccionario el elemnto : un espacio +
                # la palabra + el simbolo de fin de palabra </w>.
                # NOTA : list() aqui puede ser opciones, tal vez.
                vocab[' '.join(list(word)) + ' </w>'] += 1 
    return vocab

print(get_vocab('miniCorpus.txt'))

defaultdict(<class 'int'>, {'Y o , </w>': 1, 'J u a n </w>': 1, 'G a l l o </w>': 1, 'd e </w>': 8, 'A n d r a d a , </w>': 1, 'e s c r i b a n o </w>': 1, 'C á m a r a </w>': 1, 'd e l </w>': 4, 'R e y </w>': 1, 'n u e s t r o </w>': 1, 's e ñ o r , </w>': 1, 'l o s </w>': 2, 'q u e </w>': 6, 'r e s i d e n </w>': 1, 'e n </w>': 4, 's u </w>': 1, 'C o n s e j o , </w>': 1, 'c e r t i f i c o </w>': 1, 'y </w>': 10, 'd o y </w>': 1, 'f e </w>': 1, 'q u e , </w>': 1, 'h a b i e n d o </w>': 1, 'v i s t o </w>': 1, 'p o r </w>': 2, 's e ñ o r e s </w>': 1, 'd é l </w>': 1, 'u n </w>': 1, 'l i b r o </w>': 3, 'i n t i t u l a d o </w>': 1, 'E l </w>': 1, 'i n g e n i o s o </w>': 1, 'h i d a l g o </w>': 1, 'l a </w>': 2, 'M a n c h a , </w>': 1, 'c o m p u e s t o </w>': 1, 'M i g u e l </w>': 1, 'C e r v a n t e s </w>': 1, 'S a a v e d r a , </w>': 1, 't a s a r o n </w>': 1, 'c a d a </w>': 1, 'p l i e g o </w>': 1, 'd i c h o </w>': 4, 'a </w>': 3, 't r e s </w>': 2, 'm a r a v e d í

### get_stats(vocab)
Función que recibe un vocabulario (un diccionario con la frecuencia de cada palabra) y devuelve un diccionario con la frecuencia de los bigramas (pares de palabras consecutivas) en el vocabulario.

In [4]:
def get_stats(vocab):
    # Declara un diccionario vacío
    pairs = collections.defaultdict(int)

    # Iteramos por cada palabra y tomando su respectiva frecuencia
    for word, freq in vocab.items():
        # Separa las palabras por letra
        symbols = word.split()
        print(symbols)
        # Esto se lee "Recorre el largo de"
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

get_stats({'Y o , </w>': 1, 'J u a n </w>': 1, 'G a l l o </w>': 1, 'd e </w>': 8, 'A n d r a d a , </w>': 1, 'e s c r i b a n o </w>': 1, 'C á m a r a </w>': 1, 'd e l </w>': 4, 'R e y </w>': 1, 'n u e s t r o </w>': 1, 's e ñ o r , </w>': 1, 'l o s </w>': 2, 'q u e </w>': 6, 'r e s i d e n </w>': 1, 'e n </w>': 4, 's u </w>': 1, 'C o n s e j o , </w>': 1, 'c e r t i f i c o </w>': 1, 'y </w>': 10, 'd o y </w>': 1, 'f e </w>': 1, 'q u e , </w>': 1, 'h a b i e n d o </w>': 1, 'v i s t o </w>': 1, 'p o r </w>': 2, 's e ñ o r e s </w>': 1, 'd é l </w>': 1, 'u n </w>': 1, 'l i b r o </w>': 3, 'i n t i t u l a d o </w>': 1, 'E l </w>': 1, 'i n g e n i o s o </w>': 1, 'h i d a l g o </w>': 1, 'l a </w>': 2, 'M a n c h a , </w>': 1, 'c o m p u e s t o </w>': 1, 'M i g u e l </w>': 1, 'C e r v a n t e s </w>': 1, 'S a a v e d r a , </w>': 1, 't a s a r o n </w>': 1, 'c a d a </w>': 1, 'p l i e g o </w>': 1, 'd i c h o </w>': 4, 'a </w>': 3, 't r e s </w>': 2, 'm a r a v e d í s </w>': 2, 'm e d i o ; </w>': 1, 'e l </w>': 2, 'c u a l </w>': 1, 't i e n e </w>': 1, 'o c h e n t a </w>': 1, 'p l i e g o s , </w>': 1, 'a l </w>': 2, 'p r e c i o </w>': 2, 'm o n t a </w>': 1, 'd o c i e n t o s </w>': 1, 'n o v e n t a </w>': 1, 'm e d i o , </w>': 1, 's e </w>': 4, 'h a </w>': 1, 'v e n d e r </w>': 2, 'p a p e l ; </w>': 1, 'd i e r o n </w>': 1, 'l i c e n c i a </w>': 1, 'p a r a </w>': 2, 'e s t e </w>': 1, 'p u e d a </w>': 2, 'v e n d e r , </w>': 1, 'm a n d a r o n </w>': 1, 'e s t a </w>': 1, 't a s a </w>': 1, 'p o n g a </w>': 1, 'p r i n c i p i o </w>': 1, 'l i b r o , </w>': 1, 'n o </w>': 1, 's i n </w>': 1, 'e l l a . </w>': 1, 'Y , </w>': 1, 'd e l l o </w>': 1, 'c o n s t e , </w>': 1, 'd i </w>': 1, 'p r e s e n t e </w>': 1, 'V a l l a d o l i d , </w>': 1, 'v e i n t e </w>': 1, 'd í a s </w>': 1, 'm e s </w>': 1, 'd e c i e m b r e </w>': 1, 'm i l </w>': 1, 's e i s c i e n t o s </w>': 1, 'c u a t r o </w>': 1, 'a ñ o s . </w>': 1})

['Y', 'o', ',', '</w>']
['J', 'u', 'a', 'n', '</w>']
['G', 'a', 'l', 'l', 'o', '</w>']
['d', 'e', '</w>']
['A', 'n', 'd', 'r', 'a', 'd', 'a', ',', '</w>']
['e', 's', 'c', 'r', 'i', 'b', 'a', 'n', 'o', '</w>']
['C', 'á', 'm', 'a', 'r', 'a', '</w>']
['d', 'e', 'l', '</w>']
['R', 'e', 'y', '</w>']
['n', 'u', 'e', 's', 't', 'r', 'o', '</w>']
['s', 'e', 'ñ', 'o', 'r', ',', '</w>']
['l', 'o', 's', '</w>']
['q', 'u', 'e', '</w>']
['r', 'e', 's', 'i', 'd', 'e', 'n', '</w>']
['e', 'n', '</w>']
['s', 'u', '</w>']
['C', 'o', 'n', 's', 'e', 'j', 'o', ',', '</w>']
['c', 'e', 'r', 't', 'i', 'f', 'i', 'c', 'o', '</w>']
['y', '</w>']
['d', 'o', 'y', '</w>']
['f', 'e', '</w>']
['q', 'u', 'e', ',', '</w>']
['h', 'a', 'b', 'i', 'e', 'n', 'd', 'o', '</w>']
['v', 'i', 's', 't', 'o', '</w>']
['p', 'o', 'r', '</w>']
['s', 'e', 'ñ', 'o', 'r', 'e', 's', '</w>']
['d', 'é', 'l', '</w>']
['u', 'n', '</w>']
['l', 'i', 'b', 'r', 'o', '</w>']
['i', 'n', 't', 'i', 't', 'u', 'l', 'a', 'd', 'o', '</w>']
['E', 'l', '</w

defaultdict(int,
            {('Y', 'o'): 1,
             ('o', ','): 4,
             (',', '</w>'): 14,
             ('J', 'u'): 1,
             ('u', 'a'): 3,
             ('a', 'n'): 5,
             ('n', '</w>'): 11,
             ('G', 'a'): 1,
             ('a', 'l'): 6,
             ('l', 'l'): 4,
             ('l', 'o'): 4,
             ('o', '</w>'): 24,
             ('d', 'e'): 18,
             ('e', '</w>'): 24,
             ('A', 'n'): 1,
             ('n', 'd'): 6,
             ('d', 'r'): 2,
             ('r', 'a'): 7,
             ('a', 'd'): 4,
             ('d', 'a'): 6,
             ('a', ','): 3,
             ('e', 's'): 12,
             ('s', 'c'): 2,
             ('c', 'r'): 1,
             ('r', 'i'): 2,
             ('i', 'b'): 5,
             ('b', 'a'): 1,
             ('n', 'o'): 3,
             ('C', 'á'): 1,
             ('á', 'm'): 1,
             ('m', 'a'): 4,
             ('a', 'r'): 7,
             ('a', '</w>'): 19,
             ('e', 'l'): 10,
        

### marge_vocab(pair, v_in)

In [6]:
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

### get_tokens(vocab)

In [7]:
def get_tokens(vocab):
    tokens = collections.defaultdict(int)
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens[token] += freq
    return tokens

# Codigo principal

In [ ]:
# Minicorpus a usar :
# https://drive.google.com/file/d/17h_rLrWL2xg3jD0U1CCseeaAd6t17yc0/view?usp=share_linket 

vocab = get_vocab('miniCorpus.txt')

print('==========')
print('Tokens Before BPE')
tokens = get_tokens(vocab)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))
print('==========')

num_merges = 15
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    tokens = get_tokens(vocab)
    print('Tokens: {}'.format(tokens))
    print('Number of tokens: {}'.format(len(tokens)))
    print('==========')